In [1]:
import tensorflow as tf
import numpy as np
import matplotlib as plt
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas

i_s = 227
tf.keras.backend.clear_session()
%precision 4

L2_WEIGHT_DECAY = 0.01
L1_WEIGHT_DECAY = 0.003

In [26]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

Physical devices cannot be modified after being initialized


In [5]:
input_layer = tf.keras.layers.Input(shape=(i_s, i_s, 3))
x = tf.keras.layers.Conv2D(96, (11,11), strides=4, padding="valid", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(input_layer)
x = tf.keras.layers.Activation("relu")(x)
x = tf.keras.layers.MaxPool2D((3,3), strides=2)(x)

x = tf.keras.layers.Conv2D(256, (5,5), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
x = tf.keras.layers.Activation("relu")(x)
x = tf.keras.layers.MaxPool2D((3,3), strides=2)(x)

x = tf.keras.layers.Conv2D(384, (5,5), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
x = tf.keras.layers.Activation("relu")(x)

x = tf.keras.layers.Conv2D(384, (5,5), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
x = tf.keras.layers.Activation("relu")(x)

x = tf.keras.layers.Conv2D(256, (5,5), padding="same", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
x = tf.keras.layers.Activation("relu")(x)
x = tf.keras.layers.MaxPool2D((3,3), strides=2)(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(4096, activation="relu", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
x = tf.keras.layers.Dense(4096, activation="relu", kernel_initializer='he_normal', bias_initializer="he_normal", kernel_regularizer=regularizers.l2(L2_WEIGHT_DECAY))(x)
x = tf.keras.layers.Dense(5, activation="softmax", kernel_initializer='he_normal', bias_initializer="he_normal")(x)

model = Model(input_layer, x, name='alexNet')

model.summary()

Model: "alexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         [(None, 227, 227, 3)]     0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_15 (Activation)   (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_16 (Conv2D)           (None, 27, 27, 256)       614656    
_________________________________________________________________
activation_16 (Activation)   (None, 27, 27, 256)       0         
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 13, 13, 256)       0   

In [6]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, decay=0.0001), loss='categorical_crossentropy', metrics=['acc'])

In [7]:
check_points = "checkpoint/checkpoint_sound.hb/"
check_point_dir = os.path.dirname(check_points)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=check_point_dir, verbose=1, monitor="val_acc", save_best_only=True)